In [11]:
import scipy
import sklearn
import sklearn.cluster
import pickle
from enum import Enum

from sklearn.decomposition import PCA
import numpy as np
from sklearn.preprocessing import StandardScaler

class Objects(Enum):
    banana=1
    bulldozer=2
    chair=3
    eyeglasses=4
    flashlight=5
    foot=6
    hand=7
    harp=8
    hat=9
    keyboard=10
    laptop=11
    nose=12
    parrot=13
    penguin=14
    pig=15
    skyscraper=16
    snowman=17
    spider=18
    trombone=19
    violin=20

def create_train():
    lengths=[]
    filename="train/"+Objects(1).name+ ".npy"
    obj_np=np.load(filename)
    obj_np=np.array(obj_np/255)
    lengths.append(len(obj_np))
    for i in range(1,20):
        filename="train/"+Objects(i+1).name+ ".npy"
        temp=np.load(filename)
        temp=np.array(temp/255)
        lengths.append(len(temp))
        obj_np=np.vstack((obj_np,temp))


#     print(lengths)
    return obj_np, lengths
        
X_train, lengths=create_train()
X_train= StandardScaler().fit_transform(X_train)

In [30]:
X_conv_train=X_train.reshape(100000, 28,28,1)


In [31]:
import csv
# print(X_train)
def csv_writer(test_answers):
    with open('answers.csv', 'w') as csvfile:
        mywriter = csv.writer(csvfile)
        mywriter.writerow(["ID", "CATEGORY"])
        for i in range(len(test_answers)):
            mywriter.writerow([i, Objects(test_answers[i]+1).name])
 

In [47]:
def createYtrain():
    Y_train=[]
    for i in range(20):
        for j in range(lengths[i]):
            temp=np.zeros((1,20))
            temp[0][i]+=1
            Y_train.append(temp[0])
    return np.array(Y_train)

def createY():
    Y_train=[]
    for i in range(20):
        for j in range(lengths[i]):
            Y_train.append(i)
    return np.array(Y_train)

Y_train=createYtrain()
print(Y_train)

[[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]


In [48]:
from sklearn.model_selection import train_test_split 
for i in range(1):
    trainX, testX, trainY, testY = train_test_split(X_conv_train, Y_train, test_size=0.1, shuffle=True)


In [49]:
print(trainY.shape)


(90000, 20)


In [95]:
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten

In [112]:
model=Sequential()
model.add(Conv2D(128, kernel_size=(3, 3),activation='relu',input_shape=(28,28,1), padding='same'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(20, activation='softmax'))
# model.add(Dense(4000, input_dim=784, activation='sigmoid'))

In [113]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [115]:
model.fit(X_conv_train, Y_train, epochs=10, batch_size=100)

Epoch 1/10
100000/100000 [==============================] - 2448s 24ms/step - loss: 0.3500 - acc: 0.8935
Epoch 2/10
100000/100000 [==============================] - 1997s 20ms/step - loss: 0.2570 - acc: 0.9205
Epoch 3/10
100000/100000 [==============================] - 1144s 11ms/step - loss: 0.1706 - acc: 0.9455
Epoch 4/10
100000/100000 [==============================] - 1111s 11ms/step - loss: 0.1013 - acc: 0.9669
Epoch 5/10
100000/100000 [==============================] - 5443s 54ms/step - loss: 0.0615 - acc: 0.9799
Epoch 6/10
 80000/100000 [=======================>......] - ETA: 10:53 - loss: 0.0419 - acc: 0.9861

KeyboardInterrupt: 

In [105]:
# model.fit(X_conv_train, Y_train, epochs=5, batch_size=100)

In [62]:
model.evaluate(testX, testY)

10000/10000 [==============================] - 6s 648us/step


[0.5453157703399658, 0.8403]

In [54]:
import h5py
model.save('partially_trainedcnn.h5')

In [57]:
# model.evaluate(testX, testY)

In [61]:
from keras.models import load_model
model = load_model('partially_trainedcnn.h5')

In [110]:
# model.fit(trainX, trainY, epochs=5, batch_size=100)

In [111]:
# model.fit(trainX, trainY, epochs=5, batch_size=100)

In [64]:
def create_test():
    filename="test/test.npy"
    obj_np=np.load(filename)
    return obj_np

#     print(lengths)
X_test=create_test()
X_test= StandardScaler().fit_transform(X_test)


/usr/local/lib/python3.6/site-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype uint8 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


In [116]:
X_tst_conv=X_test.reshape(100000,28,28,1)
test_out=model.predict(X_tst_conv)

In [117]:
print(test_out)
labels=np.argmax(test_out, axis=1)
print(len(labels))


[[9.9940658e-01 2.1406779e-09 8.8493533e-09 ... 2.9121213e-07
  2.1730886e-11 2.8009489e-04]
 [1.7280658e-11 2.8589866e-13 4.4787359e-13 ... 7.7690466e-08
  9.0110028e-18 7.9636497e-20]
 [1.4490287e-16 2.4833710e-15 9.8891139e-21 ... 7.3363327e-21
  1.1574257e-12 2.3238935e-17]
 ...
 [6.1362493e-10 1.8703675e-08 9.9945492e-01 ... 1.4870503e-04
  1.8987416e-12 3.8734072e-10]
 [1.8749919e-03 2.0123466e-08 1.2318500e-13 ... 1.3541665e-10
  3.5829725e-11 2.9042029e-08]
 [2.9634899e-20 2.3054312e-15 3.6634273e-14 ... 6.3599739e-20
  7.4329372e-13 3.7020004e-16]]
100000


In [118]:
csv_writer(labels)

In [109]:
model.save('cnn_64_1024_e-15all.h5')